In [1]:
!pip install -q gradio requests
!pip install --upgrade gradio requests -q

  You can safely remove it manually.


In [ ]:
import gradio as gr
import requests
import json
from typing import Tuple, Optional
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import io
import fitz
import base64

SYMBOL_API = "https://ae9299c4f2d0.ngrok-free.app"
TEXT_API = "https://b8bbb7ea43a3.ngrok-free.app"

def generate_symbol_visualization(pdf_path, symbol_detections, dpi=300):
    """
    Generate annotated image showing detected symbols with bounding boxes.
    Uses the same visualization style as YOLO's result.plot().
    
    Args:
        pdf_path: Path to the PDF file
        symbol_detections: List of symbol detections from /detect_symbols API
        dpi: DPI for PDF rendering
    
    Returns:
        PIL Image with symbol annotations (red boxes)
    """
    try:
        # Convert PDF to image
        doc = fitz.open(pdf_path)
        page = doc[0]
        pix = page.get_pixmap(dpi=dpi)
        img_data = pix.tobytes("png")
        img = Image.open(io.BytesIO(img_data)).convert("RGB")
        doc.close()
        
        draw = ImageDraw.Draw(img)
        
        # Try to load a font, fallback to default
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()
        
        # Draw symbol detections (RED boxes)
        for detection in symbol_detections:
            if 'box_pixels' in detection:
                box = detection['box_pixels']
                x1 = int(box.get('x1', 0))
                y1 = int(box.get('y1', 0))
                x2 = int(box.get('x2', 0))
                y2 = int(box.get('y2', 0))
                
                # Draw rectangle with thick red border
                draw.rectangle([x1, y1, x2, y2], outline=(255, 0, 0), width=3)
                
                # Draw label background
                label = detection.get('label', 'Symbol')
                conf = detection.get('confidence', 0)
                text_label = f"{label} {conf:.2f}"
                
                # Get text size for background
                bbox = draw.textbbox((x1, y1), text_label, font=font)
                text_width = bbox[2] - bbox[0]
                text_height = bbox[3] - bbox[1]
                
                # Draw label background (red)
                draw.rectangle([x1, y1 - text_height - 4, x1 + text_width + 4, y1], fill=(255, 0, 0))
                # Draw label text (white on red)
                draw.text((x1 + 2, y1 - text_height - 2), text_label, fill=(255, 255, 255), font=font)
        
        return img
        
    except Exception as e:
        print(f"[ERROR] Symbol visualization failed: {e}")
        # Create error image
        error_img = Image.new('RGB', (800, 600), color=(240, 240, 240))
        draw = ImageDraw.Draw(error_img)
        draw.text((50, 280), f"Symbol Visualization Error: {str(e)}", fill=(255, 0, 0))
        return error_img


def generate_text_visualization(pdf_path, text_detections, dpi=300):
    """
    Generate annotated image showing detected text with bounding boxes.
    
    Args:
        pdf_path: Path to the PDF file
        text_detections: List of OCR results from /detect_text API
        dpi: DPI for PDF rendering
    
    Returns:
        PIL Image with text annotations (green boxes)
    """
    try:
        # Convert PDF to image
        doc = fitz.open(pdf_path)
        page = doc[0]
        pix = page.get_pixmap(dpi=dpi)
        img_data = pix.tobytes("png")
        img = Image.open(io.BytesIO(img_data)).convert("RGB")
        doc.close()
        
        draw = ImageDraw.Draw(img)
        
        # Try to load a font, fallback to default
        try:
            font = ImageFont.truetype("arial.ttf", 12)
        except:
            font = ImageFont.load_default()
        
        # Draw text detections (GREEN boxes)
        for detection in text_detections:
            if 'Bounding Box' in detection:
                bbox = detection['Bounding Box']
                x1 = int(bbox.get('xmin', 0))
                y1 = int(bbox.get('ymin', 0))
                x2 = int(bbox.get('xmax', 0))
                y2 = int(bbox.get('ymax', 0))
                
                # Draw rectangle with green border
                draw.rectangle([x1, y1, x2, y2], outline=(0, 200, 0), width=2)
                
                # Draw text content preview (truncated)
                text_content = detection.get('Text', '')[:25]
                if len(detection.get('Text', '')) > 25:
                    text_content += "..."
                
                # Get text size for background
                text_bbox = draw.textbbox((x1, y1), text_content, font=font)
                text_width = text_bbox[2] - text_bbox[0]
                text_height = text_bbox[3] - text_bbox[1]
                
                # Draw label background (green)
                draw.rectangle([x1, y1 - text_height - 4, x1 + text_width + 4, y1], fill=(0, 200, 0))
                # Draw label text (white on green)
                draw.text((x1 + 2, y1 - text_height - 2), text_content, fill=(255, 255, 255), font=font)
        
        return img
        
    except Exception as e:
        print(f"[ERROR] Text visualization failed: {e}")
        # Create error image
        error_img = Image.new('RGB', (800, 600), color=(240, 240, 240))
        draw = ImageDraw.Draw(error_img)
        draw.text((50, 280), f"Text Visualization Error: {str(e)}", fill=(0, 200, 0))
        return error_img


def validate_packaging(
    excel_file,
    pdf_file,
    product_type,
    dpi,
    packaging_width,
    packaging_height,
    confidence_threshold,
    progress=gr.Progress()
):
    """
    Complete validation pipeline - runs everything automatically!
    Returns: summary, symbol_json, text_json, symbol_viz, text_viz
    """

    if not excel_file or not pdf_file:
        return "Please upload both Excel and PDF files.", None, None, None, None

    try:
        progress(0.1, desc="Converting Excel to legal rules...")

        # Convert Excel for Symbol Detection
        with open(excel_file.name, 'rb') as f:
            response = requests.post(
                f"{SYMBOL_API}/convert_excel_logo",
                files={'excel': f},
                timeout=600
            )
        if response.status_code != 200:
            return f"Symbol Excel conversion failed: {response.json().get('message', 'Unknown error')}", None, None, None, None

        # Convert Excel for Text Detection
        with open(excel_file.name, 'rb') as f:
            response = requests.post(
                f"{TEXT_API}/convert_excel_text",
                files={'excel': f},
                timeout=600
            )
        if response.status_code != 200:
            return f"Text Excel conversion failed: {response.json().get('message', 'Unknown error')}", None, None, None, None

        progress(0.2, desc="Legal rules generated.")
        progress(0.25, desc="Loading AI models...")

        # Load Symbol Detection models
        response = requests.post(f"{SYMBOL_API}/load_models", timeout=60)
        if response.status_code != 200:
            return f"Symbol model loading failed: {response.json().get('message', 'Unknown error')}", None, None, None, None

        # Load Text Detection models
        response = requests.post(f"{TEXT_API}/load_models", timeout=180)
        if response.status_code != 200:
            return f"Text model loading failed: {response.json().get('message', 'Unknown error')}", None, None, None, None

        progress(0.35, desc="Models loaded.")
        progress(0.4, desc="Detecting symbols...")

        # Detect symbols
        with open(pdf_file.name, 'rb') as f:
            files = {'pdf': f}
            data = {
                'dpi': dpi,
                'confidence_threshold': confidence_threshold
            }
            response = requests.post(
                f"{SYMBOL_API}/detect_symbols",
                files=files,
                data=data,
                timeout=180
            )
        if response.status_code != 200:
            return f"Symbol detection failed: {response.json().get('message', 'Unknown error')}", None, None, None, None

        detections = response.json()
        symbol_detections = detections.get('detections', [])
        
        progress(0.45, desc="Generating symbol visualization...")
        
        # Generate Symbol Visualization
        symbol_viz = generate_symbol_visualization(pdf_file.name, symbol_detections, dpi)
        
        progress(0.5, desc="Validating symbols...")

        # Validate symbols
        validation_response = requests.post(
            f"{SYMBOL_API}/validate_symbols",
            json={
                'detections': symbol_detections,
                'country': "Default",
                'product_metadata': {
                    'type': product_type,
                    'width_cm': packaging_width,
                    'height_cm': packaging_height
                }
            },
            timeout=300
        )
        if validation_response.status_code != 200:
            return f"Symbol validation failed: {validation_response.json().get('message', 'Unknown error')}", None, None, symbol_viz, None

        symbol_results = {
            "status": "success",
            "detections": detections,
            "validation": validation_response.json()
        }

        # CLEAR GPU MEMORY BEFORE TEXT DETECTION
        progress(0.55, desc="Clearing GPU memory...")
        try:
            clear_response = requests.post(f"{SYMBOL_API}/clear_gpu", timeout=10)
            if clear_response.status_code == 200:
                print("[INFO] GPU memory cleared successfully")
            else:
                print("[WARN] GPU clear failed, continuing anyway")
        except Exception as e:
            print(f"[WARN] GPU clear request failed: {e}")
            pass

        # Text detection
        progress(0.6, desc="Detecting text...")
        max_retries = 3
        retry_count = 0
        response = None

        while retry_count < max_retries:
            try:
                with open(pdf_file.name, 'rb') as f:
                    files = {'pdf': f}
                    data = {
                        'dpi': dpi,
                        'detect_panels': 'false'
                    }

                    response = requests.post(
                        f"{TEXT_API}/detect_text",
                        files=files,
                        data=data,
                        timeout=180
                    )
                break

            except requests.exceptions.SSLError as e:
                retry_count += 1
                if retry_count < max_retries:
                    progress(0.6, desc=f"SSL error, retrying ({retry_count}/{max_retries})...")
                    import time
                    time.sleep(2)
                else:
                    return f"Text detection failed after {max_retries} retries due to SSL error.", None, None, symbol_viz, None

            except Exception as e:
                return f"Text detection failed: {str(e)}", None, None, symbol_viz, None

        if response is None or response.status_code != 200:
            error_msg = response.json().get('message', 'Unknown error') if response else 'No response'
            return f"Text detection failed: {error_msg}", None, None, symbol_viz, None

        ocr = response.json()
        text_detections = ocr.get('ocr_results', [])
        
        progress(0.75, desc="Generating text visualization...")
        
        # Generate Text Visualization
        text_viz = generate_text_visualization(pdf_file.name, text_detections, dpi)

        progress(0.8, desc="Validating text...")

        validation_response = requests.post(
            f"{TEXT_API}/validate_text",
            json={
                'ocr_results': text_detections,
                'country': "Default",
                'product_metadata': {
                    'type': product_type,
                    'width_cm': packaging_width,
                    'height_cm': packaging_height
                }
            },
            timeout=300
        )
        if validation_response.status_code != 200:
            return f"Text validation failed: {validation_response.json().get('message', 'Unknown error')}", None, None, symbol_viz, text_viz

        text_results = {
            "status": "success",
            "ocr": ocr,
            "validation": validation_response.json()
        }

        progress(0.95, desc="Generating report...")

        summary = format_summary(symbol_results, text_results)
        symbol_json = json.dumps(symbol_results, indent=2)
        text_json = json.dumps(text_results, indent=2)
        progress(1.0, desc="Validation complete.")

        return summary, symbol_json, text_json, symbol_viz, text_viz

    except Exception as e:
        return f"Error: {str(e)}", None, None, None, None


def format_summary(symbol_results, text_results):
    summary = "# Validation Summary\n\n"

    if symbol_results:
        symbol_val = symbol_results.get('validation', {}).get('summary', {})
        summary += "## Symbol Detection\n"
        summary += f"- Total Rules: {symbol_val.get('total_rules', 0)}\n"
        summary += f"- Passed: {symbol_val.get('passed', 0)}\n"
        summary += f"- Failed: {symbol_val.get('failed', 0)}\n"
        summary += f"- Compliance Rate: {symbol_val.get('compliance_rate', 0):.1%}\n\n"

    if text_results:
        text_val = text_results.get('validation', {}).get('summary', {})
        summary += "## Text Detection\n"
        summary += f"- Total Rules: {text_val.get('total_rules', 0)}\n"
        summary += f"- Passed: {text_val.get('passed', 0)}\n"
        summary += f"- Failed: {text_val.get('failed', 0)}\n"
        summary += f"- Compliance Rate: {text_val.get('compliance_rate', 0):.1%}\n\n"

    if symbol_results and text_results:
        s_rate = symbol_results.get('validation', {}).get('summary', {}).get('compliance_rate', 0)
        t_rate = text_results.get('validation', {}).get('summary', {}).get('compliance_rate', 0)
        overall = (s_rate + t_rate) / 2
        summary += "## Overall Status\n"
        if overall >= 0.9:
            summary += "COMPLIANT - Packaging meets requirements.\n"
        elif overall >= 0.7:
            summary += "PARTIALLY COMPLIANT - Some issues found.\n"
        else:
            summary += "NON-COMPLIANT - Multiple violations.\n"
    return summary


def create_interface():
    with gr.Blocks(title="Packaging Compliance Validator", theme=gr.themes.Monochrome()) as demo:
        gr.Markdown("""
        # DSO System
        Simple 3-Step Process:
        1. Upload your Excel file (legal requirements)
        2. Upload your packaging PDF
        3. Click "Validate" - everything runs automatically!
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## Upload Files")

                excel_file = gr.File(
                    label="Excel File (Legal Requirements)",
                    file_types=[".xlsx", ".xls"],
                    type="filepath"
                )

                pdf_file = gr.File(
                    label="Packaging PDF",
                    file_types=[".pdf"],
                    type="filepath"
                )

                gr.Markdown("## Configuration")

                product_type = gr.Dropdown(
                    choices=["5-Pack", "Single", "Multi-Pack", "Collector"],
                    value="5-Pack",
                    label="Product Type"
                )

                with gr.Accordion("Advanced Settings", open=False):
                    dpi = gr.Slider(
                        minimum=150,
                        maximum=600,
                        value=300,
                        step=50,
                        label="DPI (Image Quality)"
                    )

                    with gr.Row():
                        packaging_width = gr.Number(value=20, label="Width (cm)")
                        packaging_height = gr.Number(value=15, label="Height (cm)")

                    confidence_threshold = gr.Slider(
                        minimum=0.1,
                        maximum=0.9,
                        value=0.25,
                        step=0.05,
                        label="Detection Confidence"
                    )

                validate_btn = gr.Button("Validate Packaging", variant="primary", size="lg")

            with gr.Column(scale=2):
                gr.Markdown("## Validation Results")

                with gr.Tabs():
                    with gr.Tab("Summary"):
                        summary_output = gr.Markdown()

                    with gr.Tab("Symbol Results"):
                        symbol_output = gr.Code(language="json", label="Symbol Detection & Validation")

                    with gr.Tab("Text Results"):
                        text_output = gr.Code(language="json", label="Text Detection & Validation")

                    with gr.Tab("Symbol Visualization"):
                        symbol_viz_output = gr.Image(label="Detected Symbols (Red Boxes)")

                    with gr.Tab("Text Visualization"):
                        text_viz_output = gr.Image(label="Detected Text (Green Boxes)")

        validate_btn.click(
            fn=validate_packaging,
            inputs=[
                excel_file,
                pdf_file,
                product_type,
                dpi,
                packaging_width,
                packaging_height,
                confidence_threshold
            ],
            outputs=[summary_output, symbol_output, text_output, symbol_viz_output, text_viz_output]
        )

        gr.Markdown("""
        ---
        Notes:
        - Excel must include "Logo List - 21A" and "Text Availability - 21A" sheets.
        - Higher DPI increases quality but slows processing.
        - Model loading may take longer on first run.
        - **Symbol Visualization**: Red boxes show detected symbols/logos
        - **Text Visualization**: Green boxes show detected text blocks
        """)

    return demo


demo = create_interface()
demo.launch(
    share=True,
    server_name="0.0.0.0",
    server_port=7860,
    debug=True
)

C:\Users\user\AppData\Local\Temp\ipykernel_25996\302662868.py:232: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(title="Packaging Compliance Validator", theme=gr.themes.Monochrome()) as demo:


* Running on local URL:  http://0.0.0.0:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
